In [5]:
import cv2 as cv
import mediapipe as mp

# Initialize MediaPipe Hands module and drawing utilities
mpHands = mp.solutions.hands
drawing = mp.solutions.drawing_utils

# Open the default camera (0 = main webcam)
cam = cv.VideoCapture(0)

# Configure MediaPipe Hands detection model
hands = mpHands.Hands(
    static_image_mode=False,          # Real-time video mode
    max_num_hands=2,                  # Detect up to 2 hands
    min_detection_confidence=0.5,     # Confidence threshold for detection
    min_tracking_confidence=0.5       # Confidence threshold for tracking
)

# Landmark indices of all fingertips
finger_tips = [4, 8, 12, 16, 20]

while True:
    # Read frame from camera
    success, frame = cam.read()
    if not success:
        print("Camera not detected")
        continue

    # Mirror the image for natural viewing
    frame = cv.flip(frame, 1)

    # MediaPipe processes only RGB images (OpenCV uses BGR)
    frameRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Detect hands and landmarks in the image
    results = hands.process(frameRGB)

    # If at least one hand is detected
    if results.multi_hand_landmarks:
        # Loop through each detected hand
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):

            # Draw hand landmarks & connections on the image
            drawing.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)

            # Store all landmark coordinates (converted to pixel positions)
            lm_list = []
            h, w, c = frame.shape
            for id, lm in enumerate(hand_landmarks.landmark):
                lm_list.append((id, lm.x * w, lm.y * h))

            # Get which hand it is ("Left" or "Right")
            hand_label = results.multi_handedness[i].classification[0].label

            fingers_up = 0  # Counter for raised fingers

            # ---------------- THUMB ----------------
            # Thumb uses X-axis comparison (left-right movement)
            thumb_tip_x = lm_list[4][1]     # Landmark 4: thumb tip
            thumb_joint_x = lm_list[3][1]   # Landmark 3: thumb lower joint

            if hand_label == "Right":
                # For right hand: thumb extends leftwards
                if thumb_tip_x < thumb_joint_x:
                    fingers_up += 1
            else:  # Left hand
                # For left hand: thumb extends rightwards
                if thumb_tip_x > thumb_joint_x:
                    fingers_up += 1

            # ---------------- OTHER FOUR FINGERS ----------------
            # Compare Y-axis (up-down movement)
            for tip_id in [8, 12, 16, 20]:
                # A finger is considered "up" if tip is above the joint
                if lm_list[tip_id][2] < lm_list[tip_id - 2][2]:
                    fingers_up += 1

            # Display the result on the screen
            cv.putText(frame, f"{hand_label} Hand: {fingers_up}",
                       (10, 120 + 40 * i),
                       cv.FONT_HERSHEY_SIMPLEX, 1.2,
                       (255, 0, 0), 3)

    # Show processed video frame
    cv.imshow("Finger Counting", frame)

    # Exit loop when 'q' is pressed
    if cv.waitKey(1) == ord('q'):
        break

# Release camera and close all OpenCV windows
cam.release()
cv.destroyAllWindows()